## **Configuração do ambiente Kubernetes:**

Executor Kubernetes e criação de volumes persistentes. 

- Instalação do Minikube;
- Instalação do kubectl;

Configuração do Airflow dentro do Kubernetes: O chart do Airflow disponibilizado pelo Helm utiliza uma imagem padrão do Airflow no momento da instalação. Esta imagem vem com uma versão específica do Airflow e algumas outras bibliotecas para serem instaladas.

Para isso é necessário criar um **Dockerfile** no diretório com a versão do airflow e adicionar tb a biblioteca yfinance para fazer um teste.

Antes de executar o comando para construir a imagem, será registrado no minikube como o comando docker, utilizando o comando eval:

`eval $(minikube -p minikube docker-env)`

> Todas as vezes que executar o comando _docker_ ele será executado diretamente dentro do minikube.

Nomeando a imagem como airflow-kubernetes e utilizando . para especificar que o Dockerfie da imagem está no diretório atual:

`docker build -t airflow-kubernetes .`

Checar no repositório do Hel se o chart do Airflow foi devidamente adicionado:

`helm seach repo apache-airflow`

**O chart padrão do Helm vem com alguns serviços como: scheduler, webserver, e banco de dados Postgres; Executor Celery configurado com 1 worker; serviço de mensageria Redis; e outros serviços secundários.** Para alterar essas configurações, um novo arquivo YAML pode e deve ser criado, de acordo com a necessidade (no caso o arquivo 'override-values.yml'). Nesse arquivo estarão especificados algumas configurações que estariam no arquivo de configuração do Airflow (`airflow.cfg`) e outros relacionadas ao Kubernetes.

Executando a instalação do chart do Airflow no ambiente Kubernetes:

`minikube start`

Agora é preciso executar o comando de instalação do chart. Nele terão especificações de algumas informações, como o nome do chart (apache-airflow/airflow), o nome do namespace onde se deseja realizar a instalação (airflow) e o nome do arquivo que contém as configurações customizadas (override-values.yml):

`helm upgrade --install airflow apache-airflow/airflow --namespace airflow -f override-values.yml`

`minikube dashboard`

> Os gráficos do workload status devem estar verdinhos, que representam o status de 'run', ou seja, em execução.

Após os serviços estarem funcionando, é preciso executar o comando para expor a porta 8080:

`minikube kubectl -- port-forward svc/airflow-webserver 8080:8080 --namespace airflow`

> **OBS:** quando dar erro, usar o comando `minikube stop`; `minikube -p minikube docker-env`, depois `eval $(minikube -p minikube docker-env)`, `minikube start` e `minikube kubectl -- port-forward svc/airflow-webserver 8080:8080 --namespace airflow`.

## **Executando o DAG:**

Com o DAG pronto, é preciso **executar o minikube para ter acesso ao ambiente Airflow**. Além disso, quando executa novamente o minikube, é preciso **montar as pastas com seus volumes específicos**. É preciso realizar o processo de _mount_ antes que os pods do airflow comecem a ser executados para que seja possível acessar esses _mounts_, e o DAG que está na pastas 'dags' por meio do _mount_ com volume 'airflow-dags-volume'.

Diretório atual (do airflow-kubernetes):

`minikube mount ./dags/:/data/dags`

`minikube mount ./logs/:/data/logs`

`minikube mount ./data/:/data/data`

sudo service docker start

minikube start

minikube kubectl -- apply -f airflow-kubernetes-config.yml

where i have in the airflow-kubernetes-config.yml:

```
apiVersion: v1
kind: Namespace
metadata:
  name: airflow
  labels:
    name: airflow

---

apiVersion: v1
kind: PersistentVolume
metadata:
  name: airflow-dags-volume
  labels:
    type: local
spec:
  storageClassName: manual
  capacity:
    storage: 1Gi
  accessModes:
    - ReadWriteOnce
  hostPath:
    path: "/data/dags"

---

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: airflow-dags-claim
  namespace: airflow
spec:
  storageClassName: manual
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi

---

apiVersion: v1
kind: PersistentVolume
metadata:
  name: airflow-logs-volume
  labels:
    type: local
spec:
  storageClassName: manual
  capacity:
    storage: 1Gi
  accessModes:
    - ReadWriteOnce
  hostPath:
    path: "/data/logs"

---

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: airflow-logs-claim
  namespace: airflow
spec:
  storageClassName: manual
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi

---

apiVersion: v1
kind: PersistentVolume
metadata:
  name: airflow-data-volume
  labels:
    type: local
spec:
  storageClassName: manual
  capacity:
    storage: 1Gi
  accessModes:
    - ReadWriteOnce
  hostPath:
    path: "/data/data"

---

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: airflow-data-claim
  namespace: airflow
spec:
  storageClassName: manual
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
```

minikube mount ./dags/:/data/dags

minikube mount ./logs/:/data/logs

minikube mount ./data/:/data/data

Imagem customizada:

Dockerfile com o seguinte conteudo:

```
FROM apache/airflow:2.9.2

RUN pip install yfinance==0.1.87
```

eval $(minikube -p minikube docker-env)

docker build -t airflow-kubernetes .

helm upgrade --install airflow apache-airflow/airflow --namespace airflow -f override-values.yml

where the content of override-values.yml is:

```
executor: KubernetesExecutor
defaultAirflowRepository: airflow-kubernetes
defaultAirflowTag: latest

dags:
  persistence:
    enabled: true
    existingClaim: airflow-dags-claim

workers.replicas: 0
redis.enabled: false

securityContext:
  fsGroup: 0
  runAsGroup: 0
  runAsUser: 0

logs:
  persistence:
    enabled: true
    existingClaim: airflow-logs-claim


config:
  webserver:
    expose_config: 'True'
  core:
    parallelism: 16
```

minikube dashboard

minikube kubectl -- port-forward svc/airflow-webserver 8080:8080 --namespace airflow

minikube ssh
ls /data/dags
> check the script
exit
kubectl get pods -n airflow

kubectl exec -it -n airflow airflow-webserver-b974f6467-6qfsj -- /bin/bash
ls /opt/airflow/dags

> still not appear the dag

- Check the Persistent Volume Claims (PVCs):

kubectl get pvc -n airflow

- check persistent volume:

kubectl get pv

